<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/DLRM_DAC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn import preprocessing

In [49]:
file_path = '/content/drive/MyDrive/Colab/Learn to Rank/criteo/dac/'
columns = ['label', *(f'dense{i}' for i in range(1, 14)), *(f'sparse{i}' for i in range(1, 27))]

df_train = pd.read_csv(file_path+"1M_train.txt", sep='\t', names=columns).fillna(0)
df_test = pd.read_csv(file_path+"100k_test.txt", sep='\t', names=columns[1:]).fillna(0)
df_test

,dense1,dense2,dense3,dense4,dense5,dense6,dense7,dense8,dense9,dense10,dense11,dense12,dense13,sparse1,sparse2,sparse3,sparse4,sparse5,sparse6,sparse7,sparse8,sparse9,sparse10,sparse11,sparse12,sparse13,sparse14,sparse15,sparse16,sparse17,sparse18,sparse19,sparse20,sparse21,sparse22,sparse23,sparse24,sparse25,sparse26
0,0.0,29,50.0,5.0,7260.0,437.0,1.0,4.0,14.0,0.0,1.0,0.0,6.0,5a9ed9b0,a0e12995,a1e14474,08a40877,25c83c98,0,964d1fdd,5b392875,a73ee510,de89c3d2,59cd5ae7,8d98db20,8b216f7b,1adce6ef,78c64a1d,3ecdadf7,3486227d,1616f155,21ddcdc9,5840adea,2c277e62,0,423fab69,54c91918,9b3e8820,e75c9ae9
1,27.0,17,45.0,28.0,2.0,28.0,27.0,29.0,28.0,1.0,1.0,0.0,23.0,68fd1e64,960c983b,9fbfbfd5,38c11726,25c83c98,7e0ccccf,fe06fd10,062b5529,a73ee510,ca53fc84,67360210,895d8bbb,4f8e2224,f862f261,b4cc2435,4c0041e5,e5ba7672,b4abdd09,21ddcdc9,5840adea,36a7ab86,0,32c7478e,85e4d73f,010f6491,ee63dd9b
2,1.0,1,19.0,7.0,1.0,3.0,1.0,7.0,7.0,1.0,1.0,0.0,2.0,09ca0b81,8947f767,a87e61f7,c4ba2a67,25c83c98,7e0ccccf,ce6020cc,062b5529,a73ee510,b04d3cfe,70dcd184,899eb56b,aca22cf9,b28479f6,a473257f,88f592e4,d4bb7bd8,bd17c3da,1d04f4a4,a458ea53,82bdc0bb,0,32c7478e,5bdcd9c4,010f6491,cca57dcc
3,4.0,1,0.0,6.0,1051.0,134.0,4.0,35.0,72.0,1.0,1.0,0.0,6.0,05db9164,532da141,a7ded28e,456b4d8c,25c83c98,fbad5c96,5f29da0e,0b153874,a73ee510,4b344a42,0ad37b4b,8ea37200,f9d99d81,cfef1c29,abd8f51e,9a9902d0,07c540c4,bdc06043,0,0,6dfd157c,ad3062eb,423fab69,ef089725,0,0
4,7.0,1,25.0,10.0,139.0,74.0,48.0,13.0,44.0,1.0,8.0,2.0,12.0,05db9164,207b2d81,2b280564,ad5ffc6b,25c83c98,7e0ccccf,103c17bc,0b153874,a73ee510,8e54038a,e6e0c2dc,2a064dba,e9332a03,07d13a8f,0c67c4ca,7d9b60c8,27c07bd6,395856b0,21ddcdc9,a458ea53,9c3eb598,ad3062eb,3a171ecb,c0b8dfd6,001f3601,7a2fb9af
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0,0,112.0,5.0,1585.0,257.0,6.0,37.0,173.0,0.0,1.0,0.0,10.0,241546e0,58e67aaf,07615f7b,570a2f6f,25c83c98,7e0ccccf,f4eba758,5b392875,a73ee510,18bf508a,16faa766,000f85a5,4422e246,1adce6ef,d002b6d9,20b7c943,e5ba7672,c21c3e4c,26e97973,b1252a9d,7b16f3af,0,bcdee96c,01cc1964,9b3e8820,7b713e20
99996,4.0,-1,0.0,0.0,5.0,0.0,12.0,0.0,26.0,1.0,3.0,1.0,0.0,87552397,d4bd9877,bd840d0f,f5a1d625,25c83c98,fe6b92e5,b8a6d129,985e3fcb,a73ee510,6c47047a,0c94bc57,8066b103,03d0ba0e,d2dfe871,356fa1ec,c41d1835,8efede7f,5a87d8e9,0,0,52252402,0,3a171ecb,65d7d87d,0,0
99997,2.0,1,57.0,2.0,1467.0,81.0,4.0,2.0,102.0,0.0,1.0,0.0,2.0,05db9164,bfdcfc4a,81895bc8,b8aef670,4cf72387,7e0ccccf,59bbdf75,0b153874,a73ee510,adcd48b9,ae4c531b,b5494829,01c2bbc7,b28479f6,2ed5bdad,28a47180,e5ba7672,ffd53157,21ddcdc9,5840adea,0011318f,c9d4222a,32c7478e,1793a828,e8b83407,4e7af834
99998,4.0,56,4.0,5.0,400.0,7.0,4.0,14.0,89.0,1.0,1.0,0.0,5.0,b455c6d7,2c16a946,70b10aae,9f43a1b5,25c83c98,fbad5c96,01074d39,1f89b562,a73ee510,49023d9b,e66005d3,2fb5b882,8f33b365,b28479f6,3628a186,87140baa,27c07bd6,e4ca448c,0,0,fa77045e,c9d4222a,bcdee96c,9117a34a,0,0


In [50]:
# Preprocess Dense Features

def prepare_dense(df):
    dense_cols = [elel for elel in columns if 'dense' in elel]
    df[dense_cols] = preprocessing.StandardScaler().fit_transform(df[dense_cols])
    return df

# - - - -
df_train_ = prepare_dense(df_train)    
df_test_ = prepare_dense(df_test) 

df_train_   

,label,dense1,dense2,dense3,dense4,dense5,dense6,dense7,dense8,dense9,dense10,dense11,dense12,dense13,sparse1,sparse2,sparse3,sparse4,sparse5,sparse6,sparse7,sparse8,sparse9,sparse10,sparse11,sparse12,sparse13,sparse14,sparse15,sparse16,sparse17,sparse18,sparse19,sparse20,sparse21,sparse22,sparse23,sparse24,sparse25,sparse26
0,0,-0.124437,-0.265702,-0.036996,-0.652785,-0.250740,-0.214368,0.017562,-0.351611,0.373258,1.137014,-0.085243,-0.077480,-0.121993,68fd1e64,80e26c9b,fb936136,7b4723c4,25c83c98,7e0ccccf,de7995b8,1f89b562,a73ee510,a8cd5504,b2cb9c98,37c9c164,2824a5f6,1adce6ef,8ba8b39a,891b62e7,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,0,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,0.018833,-0.268561,0.092569,-0.527775,-0.269869,-0.204497,-0.209917,-0.351611,-0.454074,1.137014,-0.295690,-0.077480,-0.056568,68fd1e64,f0cf0024,6f67f7e5,41274cd7,25c83c98,fe6b92e5,922afcc0,0b153874,a73ee510,2b53e5fb,4f1b46f3,623049e6,d7020589,b28479f6,e6c5b5cd,c92f3b61,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,0,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,0.018833,-0.268561,-0.050284,1.097359,-0.259931,-0.004617,-0.174920,-0.351611,0.672407,1.137014,0.125203,0.954023,1.284646,287e684f,0a519c5c,02cf9876,c18be181,25c83c98,7e0ccccf,c78204a1,0b153874,a73ee510,3b08e48b,5f5e6091,8fe001f4,aa655a2f,07d13a8f,6dc710ed,36103458,8efede7f,3412118d,0,0,e587c466,ad3062eb,3a171ecb,3b183c5c,0,0
3,0,-0.267707,2.283710,-0.053606,-0.652785,-0.205756,-0.224239,-0.244914,-0.413460,-0.472771,-0.561091,-0.506136,-0.077480,-0.187418,68fd1e64,2c16a946,a9a87e68,2e17d6f6,25c83c98,fe6b92e5,2e8a689b,0b153874,a73ee510,efea433b,e51ddf94,a30567ca,3516f6e6,07d13a8f,18231224,52b8680f,1e88c74f,74ef3502,0,0,6b3a5ca6,0,3a171ecb,9117a34a,0,0
4,0,0.162103,-0.271419,-0.053606,-0.652785,-0.271363,-0.224239,-0.192418,-0.413460,-0.472771,1.137014,-0.295690,-0.077480,-0.187418,8cf07265,ae46a29d,c81688bb,f922efad,25c83c98,13718bbd,ad9fa255,0b153874,a73ee510,5282c137,e5d8af57,66a76a26,f06c53ac,1adce6ef,8ff4b403,01adbab4,1e88c74f,26b3c7a7,0,0,21c9516a,0,32c7478e,b34f3128,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0,-0.267707,-0.268561,-0.046962,-0.652785,-0.235780,-0.224239,0.000064,-0.413460,-0.379287,-0.561091,-0.295690,-0.077480,-0.187418,05db9164,287130e0,1e7e1bd9,8f8b67ad,b2241560,6f6d9be8,fb82cfe5,0b153874,a73ee510,eff5602f,9ee336c5,655bd622,094e10ad,07d13a8f,10040656,296ab802,e5ba7672,891589e7,21ddcdc9,a458ea53,14aa4334,0,32c7478e,80ff3b96,ea9a246c,1617d874
999996,0,0.018833,-0.271419,-0.036996,-0.652785,-0.254595,-0.063841,0.577510,0.947213,0.214335,-0.561091,0.125203,-0.077480,-0.187418,05db9164,1cfdf714,e2c1213d,44a6e850,25c83c98,0,78dcd87f,0b153874,a73ee510,ff61b538,8aa7f20a,b25c04d5,706a14b0,1adce6ef,f3002fbd,35879796,3486227d,e88ffc9d,b8868b4a,a458ea53,589eb146,0,423fab69,fbbe3ce2,cb079c2d,92a5e0d7
999997,1,-0.267707,-0.268561,-0.023707,0.597318,-0.212929,-0.100856,-0.104927,-0.073292,-0.290478,-0.561091,0.335649,-0.077480,0.172420,68fd1e64,6887a43c,9b792af9,9c6d05a0,25c83c98,7e0ccccf,adbcc874,0b153874,a73ee510,fa7d0797,ba515693,6532318c,377af8aa,1adce6ef,ef6b7bdf,2c9d222f,e5ba7672,8f0f692f,21ddcdc9,a458ea53,cc6a9262,ad3062eb,32c7478e,a5862ce8,445bbe3b,55fcf741
999998,1,3.457311,-0.268561,-0.050284,-0.652785,-0.271393,-0.221771,1.504923,-0.351611,-0.440052,1.137014,0.546096,-0.077480,-0.187418,05db9164,38a947a1,e39d0ad9,c5120699,25c83c98,7e0ccccf,7227c706,0b153874,a73ee510,305a0646,9625b211,dff43761,dccbd94b,1adce6ef,6c08d174,b6f5244b,e5ba7672,5a4eb947,0,0,74bd8687,0,32c7478e,1793a828,0,0


In [53]:
class CategoricalMap():
    def __init__(self, cat_cols):
        self.cat_dims = {}
        self.cat_map = {}
        self.cat_cols = cat_cols
        
    def map_categorical_train(self, df_train):
        df = df_train.copy()

        for col in self.cat_cols:
            j = 0
            my_dict = {}
            my_dict.update({'UNK':0})

            for e in df[col].value_counts().index.to_list():
                if e not in my_dict.keys():
                    j +=1
                    my_dict.update({e:j})
            self.cat_map[col] = my_dict
            self.cat_dims[col] = len(my_dict)

            df[col] = df[col].apply(lambda x: my_dict[x] if x in my_dict.keys() else my_dict['UNK'])

        return df, self.cat_dims, self.cat_map

    def map_categorical_test(self, df_test, cat_map):
        df = df_test.copy()
        for i, col in enumerate(self.cat_cols):
            df[col] = df[col].apply(lambda x: self.cat_map[col].get(x, cat_map[col]['UNK']))
        return df

cat_cols = [c for c in columns if 'sparse' in c]
preprocess_cat_feats = CategoricalMap(cat_cols)

# - - - -
df_train__, cat_dims, cat_map = preprocess_cat_feats.map_categorical_train(df_train_)
df_test__ = preprocess_cat_feats.map_categorical_test(df_test_, cat_map)

df_train__

,label,dense1,dense2,dense3,dense4,dense5,dense6,dense7,dense8,dense9,dense10,dense11,dense12,dense13,sparse1,sparse2,sparse3,sparse4,sparse5,sparse6,sparse7,sparse8,sparse9,sparse10,sparse11,sparse12,sparse13,sparse14,sparse15,sparse16,sparse17,sparse18,sparse19,sparse20,sparse21,sparse22,sparse23,sparse24,sparse25,sparse26
0,0,-0.124437,-0.265702,-0.036996,-0.652785,-0.250740,-0.214368,0.017562,-0.351611,0.373258,1.137014,-0.085243,-0.077480,-0.121993,2,16,103434,5484,1,1,3142,3,1,481,1669,89828,1478,3,317,38908,1,27,2,3,35722,1,2,1140,3,922
1,0,0.018833,-0.268561,0.092569,-0.527775,-0.269869,-0.204497,-0.209917,-0.351611,-0.454074,1.137014,-0.295690,-0.077480,-0.056568,2,18,23,30,1,2,2078,1,1,779,648,24,621,1,55,30,2,26,2,4,27,1,2,32,3,12
2,0,0.018833,-0.268561,-0.050284,1.097359,-0.259931,-0.004617,-0.174920,-0.351611,0.672407,1.137014,0.125203,0.954023,1.284646,25,51,4,1,1,1,547,1,1,1,625,5,532,2,1004,5,7,458,1,1,5,2,2,3,1,1
3,0,-0.267707,2.283710,-0.053606,-0.652785,-0.205756,-0.224239,-0.244914,-0.413460,-0.472771,-0.561091,-0.506136,-0.077480,-0.187418,2,12,226233,229,1,2,36,1,1,2,1,190844,1,2,23,148961,8,25,1,1,172317,1,2,9,1,1
4,0,0.162103,-0.271419,-0.053606,-0.652785,-0.271363,-0.224239,-0.192418,-0.413460,-0.472771,1.137014,-0.295690,-0.077480,-0.187418,4,20,226232,5,1,5,1295,1,1,1094,717,25,682,3,298,11,8,257,1,1,11,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0,-0.267707,-0.268561,-0.046962,-0.652785,-0.235780,-0.224239,0.000064,-0.413460,-0.379287,-0.561091,-0.295690,-0.077480,-0.187418,1,6,128496,3452,9,6,679,1,1,44,47,110571,52,2,9,3071,1,3,2,2,101358,1,1,496,4,487
999996,0,0.018833,-0.271419,-0.036996,-0.652785,-0.254595,-0.063841,0.577510,0.947213,0.214335,-0.561091,0.125203,-0.077480,-0.187418,1,2,128495,120965,1,4,55,1,1,5323,60,110572,67,3,4,205924,4,1,171,2,240748,1,3,41283,5,30956
999997,1,-0.267707,-0.268561,-0.023707,0.597318,-0.212929,-0.100856,-0.104927,-0.073292,-0.290478,-0.561091,0.335649,-0.077480,0.172420,2,27,21,28,1,1,439,1,1,5,781,22,159,3,136,28,1,53,2,2,25,2,1,30,7,932
999998,1,3.457311,-0.268561,-0.050284,-0.652785,-0.271393,-0.221771,1.504923,-0.351611,-0.440052,1.137014,0.546096,-0.077480,-0.187418,1,1,5194,4379,1,1,66,1,1,20,74,5399,87,3,3605,5634,1,1750,1,1,5340,1,1,5,1,1


In [54]:
df_test__

,dense1,dense2,dense3,dense4,dense5,dense6,dense7,dense8,dense9,dense10,dense11,dense12,dense13,sparse1,sparse2,sparse3,sparse4,sparse5,sparse6,sparse7,sparse8,sparse9,sparse10,sparse11,sparse12,sparse13,sparse14,sparse15,sparse16,sparse17,sparse18,sparse19,sparse20,sparse21,sparse22,sparse23,sparse24,sparse25,sparse26
0,-0.264492,-0.203966,-0.023119,-0.161000,-0.143978,0.756196,-0.238457,-0.572346,-0.387122,-0.571481,-0.306657,-0.089089,-0.113361,3,39,3237,1088,1,4,344,2,1,1646,38,3280,40,3,117,964,4,28,2,4,3306,1,3,1009,6,26
1,2.866066,-0.234961,-0.028453,2.255264,-0.257680,-0.177113,0.117815,1.027204,-0.339734,1.081727,-0.306657,-0.089089,0.636869,2,264,0,0,1,1,419,5,1,713,343,0,343,8,8453,0,1,504,2,4,0,1,1,2453,9,3566
2,-0.148545,-0.276287,-0.056190,0.049110,-0.257695,-0.234161,-0.238457,-0.380400,-0.410815,1.081727,-0.306657,-0.089089,-0.289886,14,25,1089,1420,1,1,3903,5,1,0,1858,1128,1637,1,69,1217,3,17,8,2,1181,1,1,694,9,1115
3,0.199295,-0.276287,-0.076460,-0.055945,-0.241246,0.064772,-0.197348,1.411096,-0.190802,1.081727,-0.306657,-0.089089,-0.113361,1,287,435,560,1,3,214,1,1,3202,36,449,36,5,1000,479,2,688,1,1,460,2,3,186,1,1
4,0.547134,-0.276287,-0.049789,0.364275,-0.255533,-0.072144,0.405573,0.003492,-0.285577,1.081727,0.966178,0.592385,0.151426,1,3,127,160,1,1,185,1,1,234,215,130,39,2,5,149,6,5,2,2,141,2,2,154,2,89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-0.264492,-0.278870,0.043024,-0.161000,-0.232881,0.345449,-0.169943,1.539060,0.151065,-0.571481,-0.306657,-0.089089,0.063164,13,8,89259,51400,1,1,602,2,1,3474,146,78841,167,3,27,56558,1,4,13,3,81703,1,4,2321,6,536
99996,0.199295,-0.281453,-0.076460,-0.686274,-0.257633,-0.241007,-0.087726,-0.828274,-0.346504,1.081727,0.057010,0.251648,-0.378148,7,112,492,635,1,2,245,10,1,15,1939,510,287,9,286,549,7,250,1,1,526,1,2,140,1,1
99997,-0.032598,-0.276287,-0.015651,-0.476165,-0.234729,-0.056171,-0.197348,-0.700310,-0.089257,-0.571481,-0.306657,-0.089089,-0.289886,1,101,0,8138,2,1,885,1,1,8879,187,0,201,1,291,0,1,159,2,4,0,3,1,5,3,28
99998,0.199295,-0.134228,-0.072192,-0.161000,-0.251445,-0.225033,-0.197348,0.067474,-0.133260,1.081727,-0.306657,-0.089089,-0.157492,19,12,1405,31,1,3,1312,3,1,2717,319,1451,321,1,7,31,6,12,1,1,1012,3,4,9,1,1


In [55]:
cat_dims

{'sparse1': 1262,
 'sparse2': 532,
 'sparse3': 321440,
 'sparse4': 120966,
 'sparse5': 268,
 'sparse6': 17,
 'sparse7': 10864,
 'sparse8': 564,
 'sparse9': 4,
 'sparse10': 30793,
 'sparse11': 4732,
 'sparse12': 268489,
 'sparse13': 3069,
 'sparse14': 27,
 'sparse15': 8935,
 'sparse16': 205925,
 'sparse17': 11,
 'sparse18': 3882,
 'sparse19': 1856,
 'sparse20': 5,
 'sparse21': 240749,
 'sparse22': 17,
 'sparse23': 16,
 'sparse24': 41284,
 'sparse25': 71,
 'sparse26': 30957}

In [61]:
df_train = df_train__.copy()
df_test = df_test__.copy()

In [62]:
# Num labels
df_train.label.value_counts()

0    745051
1    254949
Name: label, dtype: int64

#DLRM

In [65]:
!git clone https://github.com/facebookresearch/dlrm.git

Cloning into 'dlrm'...
remote: Enumerating objects: 642, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 642 (delta 40), reused 54 (delta 32), pack-reused 578
Receiving objects: 100% (642/642), 1.34 MiB | 17.62 MiB/s, done.
Resolving deltas: 100% (381/381), done.


In [66]:
!pip install -q captum

     |████████████████████████████████| 1.4 MB 5.3 MB/s 


In [67]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch

import sys
sys.path.insert(0, 'dlrm/')

from dlrm_s_pytorch import DLRM_Net

from captum.attr import IntegratedGradients, LayerConductance, NeuronConductance

Unable to import mlperf_logging,  No module named 'mlperf_logging'
Unable to import onnx.  No module named 'onnx'


<img src="https://drive.google.com/uc?id=1XPT6l9z8DszDDWq1cFMLVuogp4G6HmFE" alt="drawing" width="700"/>


In [81]:
len(cat_cols), len(dense_cols) #

(26, 13)

In [82]:
m_spa=len(cat_cols) # sparse feature size
ln_emb= np.array([*cat_dims.values()]) # num embed per sparse feature
ln_bot=np.array([len(dense_cols),512,256,64,16])
ln_top=np.array([367,512,256,1])

dlrm = DLRM_Net(
            m_spa,
            ln_emb,
            ln_bot,
            ln_top,
            arch_interaction_op="dot",
            arch_interaction_itself=False,
            sigmoid_bot=-1,
            sigmoid_top=ln_top.size - 2,
            sync_dense_params=True,
            loss_threshold=0.0,
            ndevices=-1,
            qr_flag=False,
            qr_operation=None,
            qr_collisions=None,
            qr_threshold=None,
            md_flag=False,
            md_threshold=None,
        )